In [ ]:
import pickle
import pandas as pd

In [ ]:
uniprot = pd.read_csv("../uniprot.tsv.gz", sep='\t').set_index("Entry")[['Protein families', "Cross-reference (Pfam)"]]
prots = pd.read_pickle("results/prepare_proteins/protein_data_whole_label_none.pkl")
prots.index = prots.index.to_series().apply(lambda x:x.split("-")[1])
prots = prots.join(uniprot)

In [ ]:
prots = prots[prots['Cross-reference (Pfam)'].notna()].reset_index().drop_duplicates("index").set_index("index")
prots.drop("Protein families", axis=1, inplace=True)

In [ ]:
prots['fam'] = prots['Cross-reference (Pfam)'].apply(lambda x: x.split(";")[0])
prots['nnodes'] = prots.data.apply(lambda x: x['x'].size(0))
prots = prots[prots['nnodes'] < 1000]

In [ ]:
prots[['data', 'fam']].to_pickle("/scratch/SCRATCH_NVME/ilya/pretrain_data/pfam_1000.pkl")